Compute DICE score between labels and segmentation volumes

In [75]:
import SimpleITK as sitk
import pandas as pd
import numpy as np
from enum import Enum
import glob

In [66]:
# Use enumerations to represent the various evaluation measures
class OverlapMeasures(Enum):
    jaccard, dice, volume_similarity, false_negative, false_positive, h_d = range(6)

In [171]:
gt_folder = '/home/robert/data/MRI_knee/GT'
seg_folder = '/home/robert/data/MRI_knee/grup_montero'
# seg_folder = '/home/robert/data/MRI_knee/grup_Penas_lopez'
# gt_folder = '/home/robert/data/MRI_knee/GT_sm'
# seg_folder = '/home/robert/data/MRI_knee/grup_barragan_molas'
# seg_folder = '/home/robert/data/MRI_knee/'
label_type = 3


In [172]:
gt_fn = sorted(glob.glob(gt_folder+'/*mhd'))
seg_fn = sorted(glob.glob(seg_folder+'/*nii'))

overlap_results = np.zeros((len(gt_fn),len(OverlapMeasures.__members__.items())))  
print(len(overlap_results))

overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
hausdorff_distance_filter = sitk.HausdorffDistanceImageFilter()

overlap_results = np.zeros((len(gt_fn),len(OverlapMeasures.__members__.items())))  

i = 0
for gt,pred in zip(gt_fn,seg_fn):
    print(gt)
    print (pred)
#     ref_im = sitk.GetArrayFromImage(sitk.ReadImage(gt))
#     seg_im = sitk.GetArrayFromImage(sitk.ReadImage(pred))
    ref_im = sitk.ReadImage(gt, sitk.sitkInt8)
    seg_im = sitk.ReadImage(pred,sitk.sitkInt8)
    
    
     # set same origins & spacings
    ref_org = ref_im.GetOrigin()   
    seg_org = seg_im.GetOrigin()
    ref_sp = ref_im.GetSpacing()
    seg_sp = seg_im.GetSpacing()

    seg_im.SetSpacing(ref_sp)
    seg_im.SetOrigin(ref_org)

    print("Pixel ID")
    print(ref_im.GetPixelIDTypeAsString(), seg_im.GetPixelIDTypeAsString())
    print("Origin")
    print(ref_org, seg_org)    
    print("Spacing")
    print(ref_sp," ",ref_sp)
    print("Size")
    print(ref_im.GetSize(), seg_im.GetSize())

    # Overlap measures
    overlap_measures_filter.Execute(ref_im, seg_im)
    overlap_results[i,OverlapMeasures.jaccard.value] = overlap_measures_filter.GetJaccardCoefficient(label_type)
    overlap_results[i,OverlapMeasures.dice.value] = overlap_measures_filter.GetDiceCoefficient(label_type)
    overlap_results[i,OverlapMeasures.volume_similarity.value] = overlap_measures_filter.GetVolumeSimilarity(label_type)
    overlap_results[i,OverlapMeasures.false_negative.value] = overlap_measures_filter.GetFalseNegativeError(label_type)
    overlap_results[i,OverlapMeasures.false_positive.value] = overlap_measures_filter.GetFalsePositiveError(label_type)
    # Hausdorff distance
    hausdorff_distance_filter.Execute(ref_im, seg_im)
    overlap_results[i,OverlapMeasures.h_d.value] = hausdorff_distance_filter.GetHausdorffDistance()
#     print(hausdorff_distance_filter.GetHausdorffDistance())
    i +=1



    

6
/home/robert/data/MRI_knee/GT/labels-080.mhd
/home/robert/data/MRI_knee/grup_montero/image-080.nii
Pixel ID
8-bit signed integer 8-bit signed integer
Origin
(13.2813, 45.3125, 46.2836) (0.0, 0.0, 0.0)
Spacing
(0.390625, 0.390625, 1.0)   (0.390625, 0.390625, 1.0)
Size
(267, 332, 104) (267, 332, 104)
/home/robert/data/MRI_knee/GT/labels-081.mhd
/home/robert/data/MRI_knee/grup_montero/image-081.nii
Pixel ID
8-bit signed integer 8-bit signed integer
Origin
(42.1848, 33.9822, -4.65878) (0.0, 0.0, 0.0)
Spacing
(0.3906, 0.3906, 1.0)   (0.3906, 0.3906, 1.0)
Size
(277, 339, 116) (277, 339, 116)
/home/robert/data/MRI_knee/GT/labels-082.mhd
/home/robert/data/MRI_knee/grup_montero/image-082.nii
Pixel ID
8-bit signed integer 8-bit signed integer
Origin
(33.201, 13.671, -109.003) (0.0, 0.0, 0.0)
Spacing
(0.3906, 0.3906, 1.0)   (0.3906, 0.3906, 1.0)
Size
(295, 437, 120) (295, 437, 120)
/home/robert/data/MRI_knee/GT/labels-083.mhd
/home/robert/data/MRI_knee/grup_montero/image-083.nii
Pixel ID
8-bit 

In [173]:
df = pd.DataFrame(overlap_results, 
                 columns =['Jaccard', 'Dice', 'Vol sim ', 'FN', 'FP', 'HD'])
df.to_csv("grup_penas_lopez.csv")
print(df)


    Jaccard      Dice  Vol sim         FN        FP         HD
0  0.559486  0.717526  0.352490  0.128957  0.389986  16.229585
1  0.682048  0.810973  0.268916  0.063046  0.285145   9.000000
2  0.585378  0.738471  0.443179  0.051309  0.395483  18.540091
3  0.497047  0.664037  0.386571  0.176863  0.443523  13.781926
4  0.666656  0.799993  0.210367  0.105970  0.276145  15.372336
5  0.496539  0.663583  0.585597  0.061678  0.486708  20.208933
